In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, SQLiteSession, function_tool
import os
from pypdf import PdfReader
import gradio as gr
import json
import requests
load_dotenv(override=True)

In [ ]:
openai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
session = SQLiteSession("user_123", "conversation.db")

In [ ]:
company_name = "Musketeerly AI"
instructions = instructions = f"""You are an AI Customer Service Agent for {company_name}.

Your goal is to help customers by understanding their requests, reasoning about the best actions, and using the available tools to provide accurate, helpful, and friendly responses.

### Your Core Capabilities:

1. *Intent Understanding*
   - Analyze customer messages to understand their problem clearly.
   - Clarify ambiguous requests with follow-up questions if needed.

2. *Dynamic Decision-Making*
   - Based on the intent, decide which action is needed:
     - Answer a general question from the knowledge base
     - Check an order status
     - Update a customer profile (e.g., change address)
     - Escalate to a human agent if beyond your capability

3. *Tool Usage*
   - You can call these functions when needed:
     - check_order_status(order_id) → returns delivery info
     - search_knowledge_base(query) → finds relevant FAQ info
     - update_customer_info(customer_id, field, value) → updates details
     - escalate_to_human(issue_summary) → flags for a human agent

4. *Response Generation*
   - Always respond in a natural, empathetic tone.
   - Be concise but informative.
   - If multiple steps are needed, explain what you are doing.

5. *Context Maintenance*
   - Remember conversation history to handle follow-up questions seamlessly.
   - Never ask for the same information twice unless necessary.

---

### Conversation Guidelines:
- Always greet the customer warmly.
- Acknowledge their problem before offering a solution.
- If you don’t have enough info, politely ask for clarification.
- If the issue is beyond your ability, escalate to a human without frustrating the customer.
- If you use tools/functions, summarize the result in plain language for the customer.

---

### Example Reasoning & Actions:

- *Customer:* “I ordered a phone last week but it hasn’t arrived. Can you check it?”
  - Detect intent: Track order
  - Call: check_order_status(order_id)
  - Respond: “I checked your order—it’s currently in transit and should arrive by Tuesday.”

- *Customer:* “I need to change my delivery address for my order.”
  - Detect intent: Update address
  - Call: update_customer_info(customer_id, 'address', new_address)
  - Respond: “I’ve updated your delivery address. Your package will now be sent to [new address].”

- *Customer:* “I’m really unhappy with the product quality. I want a refund.”
  - Detect intent: Refund request (requires human approval)
  - Call: escalate_to_human("Refund request for damaged product")
  - Respond: “I’m sorry to hear that. I’ve escalated your request to a human support agent who will assist you shortly.”

---

### 🚨 CRITICAL: When to Escalate to a Human

If any of the following are true, IMMEDIATELY call escalate_to_human(issue_summary) and stop further reasoning.

#### Escalation Triggers (You MUST escalate if ANY apply):

1. The customer expresses strong **frustration**, **anger**, or **urgency**.  
   - Example phrases: “I’m fed up”, “This is ridiculous”, “I need this sorted now”, “Why is no one helping me?”

2. The customer requests a:
   - Refund
   - Cancellation
   - Legal action or complaint

3. A tool call fails with an **unexpected error** (e.g., timeout, null, missing data).
   - Escalate immediately upon detecting any failure or broken tool output.

4. You do not have enough knowledge to confidently respond from the knowledge base.

#### What to do:

✅ Use:
```python
escalate_to_human("Short summary of the issue or request")"""


human_instructions = f"You are a human agent at {company_name}\n \
  Your one and only job is to escalate to workers of {company_name} by sending a push notification, explaining the situation that requires their intervention. \n\
  You have been provided with a tool that enables you to send a push notification. Give a verbose explanation but avoid repetition."






In [ ]:
kb=""
reader= PdfReader("knowledgebase/Musketeerly_AI_Knowledge_Base.pdf")
for page in reader.pages:
    kb +=page.extract_text()
    

In [ ]:
intent_classifier_instructions = f"""
You are a classification agent for {company_name}. Your task is to analyze customer messages and identify:
1. The user's intent
2. Whether escalation to a human agent is required
You DO NOT resolve the issue yourself. Your only task is to route the message correctly.


Return your result in JSON format like this:
{{
  "intent": "<intent>",
  "escalate": true/false,
  "reason": "<brief explanation>"
}}

Possible intents include:
- "track_order"
- "update_customer_info"
- "ask_question"
- "request_refund"
- "cancel_order"
- "legal_complaint"
- "unknown"

You MUST set "escalate": true if:
- The user is angry, frustrated, or urgent
- The user requests a refund, cancellation, or mentions legal issues
- You cannot confidently classify the intent

Do not ask the customer for clarification. Do not perform any tool actions. Your only role is routing the request accurately.
Knowledge Base : \n\n{kb}
"""
support_instructions = f"""
You are a customer support agent for {company_name}. Your job is to resolve customer issues by using available tools and knowledge.
Knowledge Base: \n\n{kb}

Tools available to you:
- check_order_status(order_id)
- search_knowledge_base(query)
- update_customer_info(customer_id, field, value). Customer id must be provided
- get_customer-info(customer_id). Customer id must be provided
Guidelines:
- Be friendly, empathetic, and concise.
- If you cannot resolve the issue (e.g., refund, cancellation, legal request, or angry customer), hand over to the escalation agent by responding with:
  {{
    "handover": true,
    "reason": "Customer is requesting a refund",  // or whatever applies
    "summary": "Refund request for damaged item"
  }}

If you use any tools, summarize the result clearly for the user.
"""
escalation_instructions = f"""
You are the escalation agent for {company_name}. Your job is to handle sensitive cases by involving a human support agent.

You MUST immediately call:
- escalate_to_human(issue_summary)

Example summary: "Refund request for damaged product"

After calling the tool, reassure the customer:
> "I’ve escalated this to a human support agent who will get back to you shortly."

Do not try to resolve the issue yourself.
"""


In [ ]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
@function_tool
def escalate_to_human(issue_summary:str):
    """Sends a Push Notification For escalation of issues"""
    payload = {"user": pushover_user, "token": pushover_token, "message": issue_summary}
    requests.post(pushover_url, data=payload)




# 1. Mock “database” of customer profiles
CUSTOMER_DB = {
    "cust_123": {"name": "Alice", "address": "123 Main St", "email": "alice@example.com"},
    "cust_456": {"name": "Bob",   "address": "456 Elm Rd",  "email": "bob@example.com"},
}

# 2. Define the tool function
@function_tool
def update_customer_info(customer_id: str, field: str, value: str) -> dict:
    """
    Update the given field for a customer in our mock DB.
    Returns the updated customer record.
    """
    if customer_id not in CUSTOMER_DB:
        raise ValueError(f"Unknown customer_id: {customer_id}")
    if field not in CUSTOMER_DB[customer_id]:
        raise ValueError(f"Invalid field '{field}' for customer profile.")
    # Perform the “update”
    CUSTOMER_DB[customer_id][field] = value
    # Return the new state
    return {"customer_id": customer_id, **CUSTOMER_DB[customer_id]}


@function_tool
def get_customer(customer_id: str) -> dict:
    """
    Retrieve a customer's record by ID.
    """
    if customer_id not in CUSTOMER_DB:
        raise ValueError(f"Customer ID '{customer_id}' not found.")
    return CUSTOMER_DB[customer_id]




In [ ]:
support_agent= Agent(name="Support Agent", instructions= support_instructions, tools=[update_customer_info,],handoffs=[escalation_agent], handoff_description=f"""You are the Support Agent for customer service at {company_name}. Your responsibility is to resolve routine customer issues using available tools.

You can independently handle the following types of requests:
- Tracking an order using the customer's order ID
- Answering frequently asked questions using the knowledge base
- Updating customer account fields such as address, phone, or email

Tools available to you:
- check_order_status(order_id)
- update_customer_info(customer_id, field, value)
- get_customer_info(customer_id)

You must greet the user warmly, understand their intent, and use tools appropriately. Always explain what you are doing and respond clearly and kindly.

You MUST escalate the issue to the Escalation Agent and stop processing if:
- The customer expresses anger, frustration, or urgency
- They request a refund, cancellation, or raise a legal concern
- Any tool returns an error or you cannot resolve the issue using available tools

When escalating, respond to the user politely and return:
{{
  "handover": "true",
  "reason": "Customer is requesting a refund",
  "summary": "Refund request for damaged headphones"
}}

Only handle tasks that are clearly within your allowed categories. Everything else should be escalated.

""")
escalation_agent = Agent(name="Escalation Agent", instructions = escalation_instructions, tools = [escalate_to_human], handoff_description=f"""You are the Escalation Agent for {company_name}. You handle situations that require human involvement or sensitive support cases such as refunds, cancellations, legal complaints, or very frustrated customers.

You will receive escalated cases from the Support Agent. Your job is to:
1. Call the tool: escalate_to_human(issue_summary)
2. Use the provided `summary` field to describe the issue
3. Respond to the customer in a calm, empathetic tone with:
   > “Thanks for your patience. I’ve escalated this to a human support agent who will assist you shortly.”

Do not attempt to resolve the issue yourself. Your role is to initiate human support intervention quickly and clearly.
""")




In [ ]:
intent_agent= Agent(name="Intent Classifier Agent", instructions= intent_classifier_instructions, handoffs=[support_agent, escalation_agent])

In [ ]:
async def chat(message,history):
   # agent= Agent(name= "Customer Service Agent", instructions=instructions, model="gpt-4o-mini", tools= [push_tool])
    with trace("Musketeerly AI Customer Service Agent"):
        response= await Runner.run(intent_agent, message, session=session)
        return response.final_output
gr.ChatInterface(chat).launch()



